In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

from sklearn.svm import SVC


In [2]:
train = pd.read_csv('/kaggle/input/signal-cluster-classification-dataset/train.csv')
test = pd.read_csv('/kaggle/input/signal-cluster-classification-dataset/test.csv')

train.head()


,sample_id,signal_strength,response_level,category
0,901,722.566585,153.933763,Group_C
1,1799,210.432525,454.613761,Group_B
2,1129,152.569777,431.993189,Group_B
3,965,670.294068,-6.559720,Group_C
4,395,49.418875,444.775273,Group_B


In [3]:
X = train.drop(['category'], axis=1)
y = train['category']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [4]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])


In [5]:
params = {
    'svm__C': [0.1, 0.5, 1, 3, 5],
    'svm__gamma': ['scale', 0.1, 0.01, 0.001],
    'svm__kernel': ['rbf']
}

grid = GridSearchCV(
    pipe,
    params,
    cv=5,
    n_jobs=-1,
    scoring='f1_macro'
)

grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best CV Score:", grid.best_score_)


Best Params: {'svm__C': 5, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Best CV Score: 0.9743962035433584


In [6]:
best_model = grid.best_estimator_
val_pred = best_model.predict(X_val)

val_score = f1_score(y_val, val_pred, average='macro')
print("Validation F1:", val_score)


Validation F1: 0.9735796559341535


In [7]:
best_model.fit(X, y)


Pipeline(steps=[('scaler', StandardScaler()), ('svm', SVC(C=5))])

In [8]:
X_test = test[X.columns]
test_pred = best_model.predict(X_test)
test_pred = le.inverse_transform(test_pred)


In [9]:
submission = pd.DataFrame({
    'sample_id': test['sample_id'],
    'category': test_pred
})

submission.to_csv('submission.csv', index=False)
submission.head()


,sample_id,category
0,1369,Group_C
1,66,Group_C
2,701,Group_B
3,939,Group_A
4,1622,Group_C
